In [ ]:
# First, you need to install the library for the Gemini API.
# Open your terminal or command prompt and run:
# pip install google-generativeai
import os 
import google.generativeai as genai

# --- Configuration ---
# IMPORTANT: Replace "YOUR_API_KEY" with the actual key you get from Google AI Studio.
# It's best practice to set this as an environment variable rather than hardcoding it.
API_KEY = os.environ['GEMINI_API_KEY'] 

try:
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring API. Make sure you have replaced 'YOUR_API_KEY'. Error: {e}")
    exit()

# --- The Few-Shot Prompt ---
# We teach the model by showing it examples. This is the core of the strategy.
prompt = """
You are an expert financial analyst. Your task is to classify a chunk of text from a company's 'Risk Factors' filing as either 'TITLE' or 'CONTENT'.

Here are some examples:

Text: 'WE DEPEND ON KEY PERSONNEL.'
Label: TITLE

Text: 'We depend on our key personnel, including Ronald L. Havner, Jr., our Chairman of the Board, and Joseph D. Russell, Jr., our President and Chief Executive Officer.'
Label: CONTENT

Text: 'We may be affected by California’s budget shortfall.'
Label: TITLE

Text: 'The California budget could affect our company in many ways, including the possible repeal of Proposition 13, which could result in higher property taxes.'
Label: CONTENT

Text: 'These risks include:'
Label: TITLE

---
Now, classify the following text. Respond with only the word 'TITLE' or 'CONTENT'.

Text: '{text_to_classify}'
Label: 
"""

def classify_chunk(text_chunk):
    """
    Uses the Gemini API with a few-shot prompt to classify a text chunk.

    Args:
        text_chunk (str): The piece of text to classify.

    Returns:
        str: The predicted label ('TITLE', 'CONTENT', or 'Error').
    """
    model = genai.GenerativeModel('gemini-pro')
    
    try:
        # Format the master prompt with the specific text we want to classify
        formatted_prompt = prompt.format(text_to_classify=text_chunk)
        
        # Make the API call
        response = model.generate_content(formatted_prompt)
        
        # The model's response should be a single word. Clean it up just in case.
        return response.text.strip()
        
    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        return "Error"

# --- Example Usage ---
if __name__ == "__main__":
    
    # These are new chunks of text the model has NOT seen in the prompt.
    test_chunks = [
        "OUR INVENTORIES ARE NOT MANAGED BY PERPETUAL INVENTORY CONTROL SYSTEMS.", # Should be TITLE
        "The systems and processes we use to manage and value our inventories require significant manual intervention.", # Should be CONTENT
        "This volatility may adversely affect the prices of our common stock regardless of our operating performance.", # Should be CONTENT
        "RISKS RELATED TO AN INVESTMENT IN OUR COMMON STOCK", # Should be TITLE
    ]

    print("--- Running AI Classification on Test Chunks ---")
    for chunk in test_chunks:
        label = classify_chunk(chunk)
        print(f"\nText:  '{chunk}'")
        print(f"AI Label: -> {label} <-")
